In [1]:
import pandas as pd
import numpy as np

import nltk
# nltk.download()
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split
# !pip install -U gensim
import gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
data = pd.read_csv("Resources/CL_data/train.csv")

In [3]:
data.head()

,Id,Category,Tweet
0,639976673685962000,0,And @whytology article 3 gives SCOTUS no legis...
1,641539752177586000,0,And 2nd it wasnt harrys fault that he had ran ...
2,639767721073315000,0,And I don't know about TGIF. Fridays are terri...
3,638850695349800000,0,"And if my day wasn't bad enough already, Snoop..."
4,638783969568366000,0,And in comes Sony with the letdown. Only one c...


In [4]:
data.rename(columns={'Id':'id',
                          'Category':'label',
                          'Tweet':'tweet'}, 
                 inplace=True)

In [5]:
data.head()

,id,label,tweet
0,639976673685962000,0,And @whytology article 3 gives SCOTUS no legis...
1,641539752177586000,0,And 2nd it wasnt harrys fault that he had ran ...
2,639767721073315000,0,And I don't know about TGIF. Fridays are terri...
3,638850695349800000,0,"And if my day wasn't bad enough already, Snoop..."
4,638783969568366000,0,And in comes Sony with the letdown. Only one c...


In [6]:
data.shape

(2742, 3)

In [7]:
data["label"].value_counts()

1    2165
0     577
Name: label, dtype: int64

## Cleaning the data

In [8]:
# dropping id column

data = data.drop("id", axis=1)

In [9]:
stopwords = stopwords.words('english')

In [10]:
def tweet_cleaner(tweet):
    tweet = re.sub(r"@\w*", " ", str(tweet).lower()).strip() #removing username
    tweet = re.sub(r'https?://[A-Za-z0-9./]+', " ", str(tweet).lower()).strip() #removing links
    tweet = re.sub(r'[^a-zA-Z]', " ", str(tweet).lower()).strip() #removing sp_char
    tw = []
    
    for text in tweet.split():
        if text not in stopwords:
            tw.append(text)
    
    return " ".join(tw)

In [11]:
data.tweet = data.tweet.apply(lambda x: tweet_cleaner(x))

### word2vec

In [12]:
documents = [text.split() for text in data.tweet]

In [13]:
len(documents)

2742

In [14]:
w2v_model = gensim.models.word2vec.Word2Vec(size = 256, window = 7, min_count = 5)

In [15]:
w2v_model.build_vocab(documents)

In [16]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(593893, 967424)

In [82]:
w2v_model.wv["sad"]

array([ 0.05213831, -0.13643545, -0.12848118, -0.03137036, -0.16733742,
       -0.35844222,  0.18349892,  0.24100281, -0.10513192, -0.00084092,
       -0.09382115,  0.0190608 ,  0.01253311, -0.19807175, -0.25202316,
        0.08577892,  0.00742784,  0.03299676, -0.00901942,  0.01562997,
       -0.01876138,  0.0836402 , -0.14236046,  0.10583993, -0.03907061,
        0.03915321, -0.03240407,  0.06672715, -0.09526563, -0.20083658,
       -0.10953088,  0.01815526,  0.0451845 ,  0.1786638 ,  0.10244184,
       -0.15505786,  0.12966278, -0.17827111,  0.1243183 , -0.13932452,
       -0.04473007, -0.1583606 , -0.06381836,  0.06500481,  0.00120833,
       -0.01642324, -0.2556382 , -0.09293447, -0.24231493,  0.07267169,
        0.01220249,  0.08600025, -0.15921576, -0.3607427 , -0.09505413,
        0.04382204,  0.09672338, -0.12200074,  0.12219553,  0.09155581,
       -0.17918134,  0.11814304, -0.08707627,  0.0129184 , -0.07477812,
        0.35935998, -0.18669072,  0.03555908, -0.04397948,  0.21

### Converting tweets to vectors

In [83]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.tweet)

In [84]:
len(tokenizer.word_index)

7011

In [85]:
x_train = pad_sequences(tokenizer.texts_to_sequences(data.tweet), maxlen=256, padding="post", truncating="post")

In [86]:
x_train

array([[1045, 1046,  363, ...,    0,    0,    0],
       [  28, 1779, 2769, ...,    0,    0,    0],
       [  33,   37, 2772, ...,    0,    0,    0],
       ...,
       [ 110,  220,    2, ...,    0,    0,    0],
       [ 110, 1262,    2, ...,    0,    0,    0],
       [ 110, 7009, 7010, ...,    0,    0,    0]], dtype=int32)

In [87]:
y_train = data.label

y_train_f = []
for x in y_train:
    if x == 1:
        y_train_f.append(1)
    elif x == 0:
        y_train_f.append(0)
    elif x == -1:
        y_train_f.append(2)
        
y_train_f = to_categorical(y_train_f)

In [88]:
y_train_f

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

### Model

In [89]:
embedding_matrix = np.zeros((14850,256))

In [90]:
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [91]:
embedding_layer = Embedding(14850, 256, weights=[embedding_matrix], input_length=256, trainable=False)

In [92]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.25))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(50, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [93]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 256)          3801600   
_________________________________________________________________
dropout_7 (Dropout)          (None, 256, 256)          0         
_________________________________________________________________
dense_12 (Dense)             (None, 256, 200)          51400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256, 200)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_13 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_14 (Dense)             (None, 100)               5100      
__________

In [94]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [95]:
model.fit(x_train, y_train_f, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 2467 samples, validate on 275 samples
Epoch 1/10
2467/2467 [==============================] - 32s 13ms/step - loss: 0.5633 - acc: 0.7588 - val_loss: 0.2721 - val_acc: 1.0000
Epoch 2/10
2467/2467 [==============================] - 30s 12ms/step - loss: 0.5493 - acc: 0.7661 - val_loss: 0.3075 - val_acc: 1.0000
Epoch 3/10
2467/2467 [==============================] - 30s 12ms/step - loss: 0.5447 - acc: 0.7661 - val_loss: 0.2193 - val_acc: 1.0000
Epoch 4/10
2467/2467 [==============================] - 32s 13ms/step - loss: 0.5465 - acc: 0.7661 - val_loss: 0.2733 - val_acc: 1.0000
Epoch 5/10
2467/2467 [==============================] - 32s 13ms/step - loss: 0.5480 - acc: 0.7661 - val_loss: 0.2894 - val_acc: 1.0000
Epoch 6/10
2467/2467 [==============================] - 31s 13ms/step - loss: 0.5481 - acc: 0.7661 - val_loss: 0.3198 - val_acc: 1.0000
Epoch 7/10
2467/2467 [==============================] - 31s 13ms/step - loss: 0.5464 - acc: 0.7661 - val_loss: 0.2948 - val_acc: 1.0000
E

# Testing

In [107]:
def sentiments(text):
    
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=256)
    score = model.predict([x_test])[0]
    
    final = "Positive = %f ,negative = %f" % (score[1], score[0])
    return print(final)

Positive = 0.746720 ,negative = 0.253280


In [98]:
model.save("twitter_model_trained.h5")